*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

## Movielens データセットでのリーマン低ランク行列補完アルゴリズム

リーマン低ランク行列補完 (RLRMC)は、リーマン共役勾配アルゴリズム (Absil et al., 2008) を用いて最適化問題を解決する行列分解法(バニラ)行列補完アルゴリズムである。RLRMCは、Jawanpuria と Mishra (2018) と Mishra et al. (2013) の論文に基づいています。

映画 (アイテム) とユーザーの評価マトリックスは、低ランクのマトリックスとしてモデル化されます。映画の数を $d$、 ユーザーの数を $T$ にしましょう。RLRMC アルゴリズムは、評価行列 $M$ (大きさは $d\回数 T$) が部分的に既知であると仮定します。$M(i,j)$ のエントリは、$i$-th ムービーに対して $j$-th ユーザーによって与えられた評価を表します。$M=LR^\top$ は RLRMC 学習行列 $M$ であり、$L$ は $d\回数 r$ 行列で、$R$ は $T\回数 r$ となります。ここでは、$r$ は RLRMCアルゴリズムに提供する必要がある順位ハイパーパラメータです。通常、$r\ll d,T$ であると仮定されます。最適化問題は、リーマン共役勾配アルゴリズムを使用して反復的に解決されます。リーマン最適化フレームワークは、共役勾配、信頼領域などのユークリッドの一次アルゴリズムと二次アルゴリズムを、とりわけリーマン多様体に汎化します。リーマン最適化フレームワークの詳細な説明は、Absil et al. (2008) で確認できます。

このノートブックでは、**reco_utils** を利用した RLRMC 実装を利用して評価する方法の例を示します。

In [1]:
import numpy as np
import sys
import time
import pandas as pd
sys.path.append("../../")
sys.path.append("../../reco_utils/recommender/rlrmc/")

from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.dataset import movielens
from reco_utils.recommender.rlrmc.RLRMCdataset import RLRMCdataset 
from reco_utils.recommender.rlrmc.RLRMCalgorithm import RLRMCalgorithm 
# Pymanopt のインストールは以下の内容の実行が必要です
# pip install pymanopt 
from reco_utils.evaluation.python_evaluation import (
    rmse, mae
)

# import logging

# %load_ext autoreload
# %autoreload 2

In [2]:
print("Pandas version: {}".format(pd.__version__))
print("System version: {}".format(sys.version))


Pandas version: 0.23.4
System version: 3.7.1 (default, Dec 14 2018, 13:28:58) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


デフォルトパラメータを設定します。


In [3]:
# Movielens のデータサイズを選択: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '10m'

# モデル パラメータ

# 正の整数(通常は小さい)を持つモデルのランク。必須パラメータです
rank_parameter = 10
# 正の数 (通常は小さい) を持つ損失関数に乗算された正規化パラメータ。必須パラメータです
regularization_parameter = 0.001
# 'svd' 単数値分解を採用した、モデルの初期化オプション。オプションのパラメータです
initialization_flag = 'svd' #default is 'random'
# 正の整数を持つソルバーの最大反復回数。オプションのパラメータです
maximum_iteration = 100 #optional, default is 100
# 正の整数を持つソルバーの最大時間 (秒)。オプションのパラメータです
maximum_time = 300#optional, default is 1000

# 中間結果の詳細度
verbosity=0 #オプションのパラメータであり、有効な値は 0,1,2、デフォルトは 0 です
# 反復単位のトレーニング RMSE を計算するかどうか (およびテスト データが与えられている場合は RMSE をテストする)
compute_iter_rmse=True #オプションのパラメータであり、ブール値、デフォルトは False です

In [4]:
## ロギング ユーティリティ。次のコマンドを使用するためには 'logging' をインポートしてください。
# logging.basicConfig(level=logging.INFO)

### 1. MovieLens データセットをダウンロードする


In [5]:

df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)

65.6MB [00:25, 2.57MB/s]                            


### 2. ユーティリティで提供されている Spark の時系列分割器を使用してデータを分割する

In [6]:
## 検証とテストセットの両方が必要な場合
# train, validation, test = python_random_split(df,[0.6, 0.2, 0.2])

## 検証セットが必要ない場合
train, test = python_random_split(df,[0.8, 0.2])

## テストセットが不要な場合
# train, validation = python_random_split(df,[0.8, 0.2])

## 検証セットとテストセットの両方が必要ない場合(つまり、完全なデータセットはモデルのトレーニング用)
# train = df

データ サブセットから RLRMCdataset オブジェクトを生成します。

In [7]:
# data = RLRMCdataset(train=train, validation=validation, test=test)
data = RLRMCdataset(train=train, test=test) # No validation set
# data = RLRMCdataset(train=train, validation=validation) # No test set
# data = RLRMCdataset(train=train) # No validation or test set

### 3. トレーニング データでの RLRMC モデルのトレーニング

In [8]:
model = RLRMCalgorithm(rank = rank_parameter,
                       C = regularization_parameter,
                       model_param = data.model_param,
                       initialize_flag = initialization_flag,
                       maxiter=maximum_iteration,
                       max_time=maximum_time)

In [9]:
start_time = time.time()

model.fit(data,verbosity=verbosity)

# fit_and_evaluateは、すべての反復で検証セット(指定されている場合)で RMSE を計算します
# model.fit_and_evaluate(data,verbosity=verbosity)

train_time = time.time() - start_time # train_time には、モデルの初期化とモデルのトレーニング時間の両方が含まれます。

print("Took {} seconds for training.".format(train_time))

Took 44.991251945495605 seconds for training.


### 4. テストデータで RLRMC モデルから予測を取得する

In [10]:
## Movielens 10m データセットから (userID,itemID) ペア (60586,54775) と (52681,36519) の予測を取得する
# output = model.predict([60586,52681],[54775,36519]) # Movielens 10m データセット

# 完全なテスト セットの予測を取得する
predictions_ndarr = model.predict(test['userID'].values,test['itemID'].values)

### 5. RLRMC のパフォーマンスを評価する

In [12]:
predictions_df = pd.DataFrame(data={"userID": test['userID'].values, "itemID":test['itemID'].values, "prediction":predictions_ndarr})

## RMSE の test を計算する
eval_rmse = rmse(test, predictions_df)
## MAE の test を計算する
eval_mae = mae(test, predictions_df)

print("RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae, sep='\n')

RMSE:	0.809386
MAE:	0.620971


### Reference
[1] Pratik Jawanpuria and Bamdev Mishra. *A unified framework for structured low-rank matrix learning*. In International Conference on Machine Learning, 2018.

[2] Bamdev Mishra, Gilles Meyer, Francis Bach, and Rodolphe Sepulchre. *Low-rank optimization with trace norm penalty*. In SIAM Journal on Optimization 23(4):2124-2149, 2013.

[3] James Townsend, Niklas Koep, and Sebastian Weichwald. *Pymanopt: A Python Toolbox for Optimization on Manifolds using Automatic Differentiation*. In Journal of Machine Learning Research 17(137):1-5, 2016.

[4] P.-A. Absil, R. Mahony, and R. Sepulchre. *Optimization Algorithms on Matrix Manifolds*. Princeton University Press, Princeton, NJ, 2008.

[5] A. Edelman, T. Arias, and S. Smith. *The geometry of algo- rithms with orthogonality constraints*. SIAM Journal on Matrix Analysis and Applications, 20(2):303–353, 1998.